In [3]:
from pyspark.sql import SparkSession

# Spark + Iceberg 설정
spark = SparkSession.builder \
    .appName("IcebergBlogDemo") \
    .config("spark.jars", "/home/jovyan/iceberg-spark.jar") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.demo", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.demo.type", "hadoop") \
    .config("spark.sql.catalog.demo.warehouse", "file:///home/jovyan/data/warehouse") \
    .getOrCreate()

print("✅ Spark + Iceberg 설정 완료!")

# 카탈로그 확인
spark.sql("SHOW CATALOGS").show()

✅ Spark + Iceberg 설정 완료!
+-------------+
|      catalog|
+-------------+
|         demo|
|spark_catalog|
+-------------+



In [4]:
# 데이터베이스 생성
spark.sql("CREATE DATABASE IF NOT EXISTS demo.blog")
print("✅ 데이터베이스 생성 완료!")

# 테이블 생성
spark.sql("""
    CREATE TABLE IF NOT EXISTS demo.blog.orders (
        order_id BIGINT,
        customer_id BIGINT,
        product_name STRING,
        order_date DATE,
        amount DECIMAL(10,2),
        status STRING
    ) USING ICEBERG
    PARTITIONED BY (order_date)
""")

print("✅ 테이블 생성 완료!")

# 확인
spark.sql("SHOW TABLES IN demo.blog").show()

✅ 데이터베이스 생성 완료!
✅ 테이블 생성 완료!
+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|     blog|   orders|      false|
+---------+---------+-----------+



In [7]:
import pandas as pd
import random
from datetime import datetime, timedelta

# 랜덤 데이터 생성
def generate_sample_data(num_records=100):
    # 상품 목록
    products = [
        'iPhone 14', 'iPhone 14 Pro', 'MacBook Pro', 'MacBook Air', 'iPad Air', 
        'iPad Pro', 'AirPods', 'AirPods Pro', 'Apple Watch', 'Magic Mouse',
        'Magic Keyboard', 'iMac', 'Mac Studio', 'Apple TV', 'HomePod',
        'iPhone 13', 'iPhone SE', 'Mac Mini', 'Studio Display', 'Pro Display'
    ]
    
    # 가격 매핑
    prices = {
        'iPhone 14': 999.99, 'iPhone 14 Pro': 1199.99, 'MacBook Pro': 1999.99,
        'MacBook Air': 1299.99, 'iPad Air': 649.99, 'iPad Pro': 899.99,
        'AirPods': 199.99, 'AirPods Pro': 299.99, 'Apple Watch': 399.99,
        'Magic Mouse': 99.99, 'Magic Keyboard': 199.99, 'iMac': 1499.99,
        'Mac Studio': 2199.99, 'Apple TV': 149.99, 'HomePod': 349.99,
        'iPhone 13': 799.99, 'iPhone SE': 449.99, 'Mac Mini': 799.99,
        'Studio Display': 1799.99, 'Pro Display': 5999.99
    }
    
    # 상태 목록
    statuses = ['completed', 'pending', 'shipped', 'cancelled', 'processing']
    
    # 날짜 범위 (3개월)
    start_date = datetime(2023, 1, 1)
    end_date = datetime(2023, 3, 31)
    
    data = []
    for i in range(1, num_records + 1):
        # 랜덤 날짜 생성
        random_days = random.randint(0, (end_date - start_date).days)
        order_date = start_date + timedelta(days=random_days)
        
        # 랜덤 상품
        product = random.choice(products)
        base_price = prices[product]
        
        # 가격에 약간의 변동 추가 (할인/할증)
        price_variation = random.uniform(0.8, 1.2)
        final_price = round(base_price * price_variation, 2)
        
        data.append({
            'order_id': i,
            'customer_id': random.randint(100, 999),
            'product_name': product,
            'order_date': order_date.strftime('%Y-%m-%d'),
            'amount': final_price,
            'status': random.choice(statuses)
        })
    
    return data

# 100개 데이터 생성
print("🎲 랜덤 데이터 100개 생성 중...")
sample_data = generate_sample_data(100)

# Pandas DataFrame 생성
df_pandas = pd.DataFrame(sample_data)

print("✅ 데이터 생성 완료!")
print(f"📊 총 레코드 수: {len(df_pandas)}")
print(f"📅 날짜 범위: {df_pandas['order_date'].min()} ~ {df_pandas['order_date'].max()}")
print(f"🛍️ 상품 종류: {df_pandas['product_name'].nunique()}개")
print(f"👥 고객 수: {df_pandas['customer_id'].nunique()}명")

# 샘플 미리보기
print("\n📋 샘플 데이터 미리보기:")
print(df_pandas.head(10))

🎲 랜덤 데이터 100개 생성 중...
✅ 데이터 생성 완료!
📊 총 레코드 수: 100
📅 날짜 범위: 2023-01-01 ~ 2023-03-31
🛍️ 상품 종류: 20개
👥 고객 수: 93명

📋 샘플 데이터 미리보기:
   order_id  customer_id product_name  order_date   amount      status
0         1          986      HomePod  2023-03-19   353.81   cancelled
1         2          165    iPhone 13  2023-02-19   941.02   completed
2         3          515    iPhone SE  2023-02-07   538.77   completed
3         4          166     iPad Pro  2023-03-02   782.19   cancelled
4         5          731    iPhone 13  2023-02-07   957.03   completed
5         6          942      AirPods  2023-02-27   237.44     pending
6         7          506    iPhone SE  2023-03-14   455.62   completed
7         8          364  Pro Display  2023-02-02  5084.19  processing
8         9          102  Magic Mouse  2023-02-22    89.64   completed
9        10          152  Pro Display  2023-02-19  6660.17     shipped


In [8]:
from pyspark.sql.functions import col

# Spark DataFrame으로 변환
df_spark = spark.createDataFrame(df_pandas)

# 데이터 타입 변환
df_spark = df_spark.withColumn("order_date", col("order_date").cast("date"))

print("📊 Spark DataFrame 스키마:")
df_spark.printSchema()

# 기존 데이터 삭제 (필요시)
spark.sql("DELETE FROM demo.blog.orders")

# 새 데이터 삽입
df_spark.write \
    .format("iceberg") \
    .mode("append") \
    .saveAsTable("demo.blog.orders")

print("✅ 100개 데이터 삽입 완료!")

# 결과 확인
print("\n📈 삽입 결과:")
spark.sql("SELECT COUNT(*) as total_orders FROM demo.blog.orders").show()

# 월별 분포 확인
spark.sql("""
    SELECT 
        DATE_FORMAT(order_date, 'yyyy-MM') as month,
        COUNT(*) as order_count,
        ROUND(SUM(amount), 2) as total_amount
    FROM demo.blog.orders 
    GROUP BY DATE_FORMAT(order_date, 'yyyy-MM')
    ORDER BY month
""").show()

📊 Spark DataFrame 스키마:
root
 |-- order_id: long (nullable = true)
 |-- customer_id: long (nullable = true)
 |-- product_name: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- amount: double (nullable = true)
 |-- status: string (nullable = true)

✅ 100개 데이터 삽입 완료!

📈 삽입 결과:
+------------+
|total_orders|
+------------+
|         100|
+------------+

+-------+-----------+------------+
|  month|order_count|total_amount|
+-------+-----------+------------+
|2023-01|         27|    28530.90|
|2023-02|         27|    39775.69|
|2023-03|         46|    46657.86|
+-------+-----------+------------+



In [9]:
import os

def show_detailed_tree(path, prefix="", max_depth=4, current_depth=0):
    if current_depth >= max_depth:
        return
    
    try:
        items = sorted([item for item in os.listdir(path) if not item.startswith('.')])
        for i, item in enumerate(items):
            item_path = os.path.join(path, item)
            is_last = i == len(items) - 1
            
            # 파일 크기 정보 추가
            size_info = ""
            if os.path.isfile(item_path):
                size = os.path.getsize(item_path)
                if size > 1024 * 1024:
                    size_info = f" ({size / 1024 / 1024:.1f} MB)"
                elif size > 1024:
                    size_info = f" ({size / 1024:.1f} KB)"
                else:
                    size_info = f" ({size} bytes)"
            
            print(f"{prefix}{'└── ' if is_last else '├── '}{item}{size_info}")
            
            if os.path.isdir(item_path):
                next_prefix = prefix + ("    " if is_last else "│   ")
                show_detailed_tree(item_path, next_prefix, max_depth, current_depth + 1)
    except PermissionError:
        pass

print("📁 100개 데이터 삽입 후 파일 구조:")
warehouse_path = "/home/jovyan/data/warehouse"
show_detailed_tree(warehouse_path)

📁 100개 데이터 삽입 후 파일 구조:
└── blog
    └── orders
        ├── data
        │   ├── order_date=2023-01-01
        │   ├── order_date=2023-01-08
        │   ├── order_date=2023-01-09
        │   ├── order_date=2023-01-11
        │   ├── order_date=2023-01-14
        │   ├── order_date=2023-01-15
        │   ├── order_date=2023-01-16
        │   ├── order_date=2023-01-17
        │   ├── order_date=2023-01-18
        │   ├── order_date=2023-01-19
        │   ├── order_date=2023-01-20
        │   ├── order_date=2023-01-21
        │   ├── order_date=2023-01-22
        │   ├── order_date=2023-01-24
        │   ├── order_date=2023-01-25
        │   ├── order_date=2023-01-26
        │   ├── order_date=2023-01-27
        │   ├── order_date=2023-01-28
        │   ├── order_date=2023-01-29
        │   ├── order_date=2023-01-30
        │   ├── order_date=2023-01-31
        │   ├── order_date=2023-02-01
        │   ├── order_date=2023-02-02
        │   ├── order_date=2023-02-04
        │   ├── order_da

In [10]:
# 상위 상품 분석
print("🛍️ 인기 상품 TOP 10:")
spark.sql("""
    SELECT 
        product_name,
        COUNT(*) as order_count,
        ROUND(AVG(amount), 2) as avg_price,
        ROUND(SUM(amount), 2) as total_revenue
    FROM demo.blog.orders 
    GROUP BY product_name
    ORDER BY order_count DESC
    LIMIT 10
""").show()

# 일별 주문 현황
print("\n📅 최근 10일 주문 현황:")
spark.sql("""
    SELECT 
        order_date,
        COUNT(*) as orders,
        ROUND(SUM(amount), 2) as revenue
    FROM demo.blog.orders 
    GROUP BY order_date
    ORDER BY order_date DESC
    LIMIT 10
""").show()

# 파티션별 파일 정보
print("\n🗂️ 파티션별 정보:")
spark.sql("""
    SELECT 
        DATE_FORMAT(order_date, 'yyyy-MM') as partition_month,
        COUNT(*) as record_count
    FROM demo.blog.orders 
    GROUP BY DATE_FORMAT(order_date, 'yyyy-MM')
    ORDER BY partition_month
""").show()

🛍️ 인기 상품 TOP 10:
+--------------+-----------+---------+-------------+
|  product_name|order_count|avg_price|total_revenue|
+--------------+-----------+---------+-------------+
|Studio Display|         10|  1925.70|     19256.98|
|       HomePod|          9|   344.33|      3098.94|
|   MacBook Air|          8|  1283.51|     10268.07|
|Magic Keyboard|          6|   213.53|      1281.17|
|      Mac Mini|          6|   735.51|      4413.06|
|     iPhone SE|          6|   477.67|      2866.03|
|       AirPods|          6|   209.44|      1256.64|
|     iPhone 13|          6|   868.72|      5212.34|
|   Pro Display|          5|  5755.54|     28777.69|
|      iPad Pro|          5|   881.49|      4407.44|
+--------------+-----------+---------+-------------+


📅 최근 10일 주문 현황:
+----------+------+-------+
|order_date|orders|revenue|
+----------+------+-------+
|2023-03-31|     2|2618.96|
|2023-03-30|     2|1708.35|
|2023-03-29|     2|2467.18|
|2023-03-27|     3|9165.16|
|2023-03-26|     2|1375.61|